In [391]:
import numpy as np
import torch
import torch.nn as nn
import cvxpy as cp

In [392]:
# generate some random channels -> just randomly generated with variance 1

# Number of samples for each channel
num_samples = 1
P_t = torch.tensor(21.0)

# Mean and variance
mean = 0  
# Assuming the mean is 0 for all channels

variance = torch.tensor(1, dtype = torch.float32)  
# Variance is set to 1 for all channels

# Generate the channels

channels = torch.rand(6)

In [409]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTM, self).__init__()

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

        # Fully connected layer
        self.linear = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # Forward through LSTM layer
        # x shape: (batch, seq_len, input_size), here seq_len is 1
        lstm_out, hidden = self.lstm(x)

        # Forward through linear layer
        # Reshape the output to (batch, output_size)
        out = self.linear(lstm_out.reshape(x.shape[0], -1))
        out = self.tanh(out)

        return out

class MyLSTM_p(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTM_p, self).__init__()

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

        # Fully connected layer
        self.linear = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Forward through LSTM layer
        # x shape: (batch, seq_len, input_size), here seq_len is 1
        lstm_out, hidden = self.lstm(x)

        # Forward through linear layer
        # Reshape the output to (batch, output_size)
        out = self.linear(lstm_out)
        out = self.sigmoid(out)

        return out


In [410]:
import torch

class CustomActivationFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)

        # Apply the piecewise function
        output = torch.where(input < -1, -2 * input - 1, 
                             torch.where(input > 1, 2 * input, input**2))
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors

        # Compute the gradient for each piece
        grad_input = torch.where(input < -1, -2 * torch.ones_like(input),
                                 torch.where(input > 1, 2 * torch.ones_like(input), 2 * input))
        return grad_input * grad_output

# To apply the custom activation function
def projection(input):
    return CustomActivationFunction.apply(input)

In [411]:
def lagrangian_eq(
    h, 
    p, 
    P_t,
    lambda_,
    variance = 1,
):
    '''### - First calculate the lagrangian equation.

    ### - Then do backward:
     - calculate the gradient for both subsequently in two networks
     - grad P | grad Lambda are the return of the function
    '''
    
    _lam = projection(lambda_)
    _L = torch.log2(1 + torch.dot(h, p)) - _lam * (torch.sum(p) - P_t)
    # do backward, calculate gradient
    _L.backward()

    return p.grad, lambda_.grad

In [412]:
# initialisation of the problem

lstm_lambda = MyLSTM(1, 20, 1)
lstm_x = MyLSTM_p(6, 20, 6)
optimiser_lambda = torch.optim.Adam(
    lstm_lambda.parameters(),
    lr = 1e-3
)

optimiser_p = torch.optim.Adam(
    lstm_x.parameters(),
    lr = 1e-3
)

In [416]:
# initialise the parameters
_p = torch.rand(6, requires_grad = True)
_lambda = torch.rand(1, requires_grad=True)
# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)
for epoch in range (100):

    # here _p is the proposed optimised variable, and will iterate with _lambda
    P_grad, Lambda_grad = lagrangian_eq(channels, _p, P_t, _lambda)

    # first iterate lambda
    _lambda_iteration = lstm_lambda(Lambda_grad.reshape(1,1,1)).squeeze()
    _lambda_next = projection(_lambda + _lambda_iteration)

    loss_lambda = torch.log2(
        1 + torch.dot(channels, _p)
    ) - _lambda_next * (torch.sum(_p) - P_t)
    
    print (loss_lambda)

    loss_lambda.backward()
    optimiser_lambda.step()

    _lambda = projection(_lambda)
    P_grad, Lambda_grad = lagrangian_eq(channels, _p, P_t, _lambda)
    _P_iteration = lstm_x(P_grad.reshape(1,1,6)).squeeze()
    _p_next = _p + _P_iteration
    loss_p = - torch.log2(
        1 + torch.dot(channels, _p_next)
    ) - _lambda_next * (torch.sum(_p_next) - P_t)

    loss_p.backward()
    optimiser_p.step()

    _p = _p_next.clone()
    _lambda = _lambda_next.clone()

print (_p)

tensor([8.9296], grad_fn=<SubBackward0>)


/opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad
/opt/anaconda3/lib/python3.9/site-packages/torch/autograd/__init__.py:173: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
 

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 6]] is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [374]:
p = cp.Variable(6)

# Define the objective function
objective = cp.Maximize(
    cp.sum(cp.log1p(cp.multiply(channels.squeeze(), p))) / np.log(2)
)


# Define the constraints
constraints = [p >= 0, cp.sum(p) <= 1]

# Define the problem and solve it
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS)

# Print the optimal power allocation
print("The optimal power allocation is:")
print(p.value)

The optimal power allocation is:
[1.27158871e-09 5.66728855e-01 1.86959056e-02 6.44981527e-08
 2.09707023e-08 4.14575151e-01]
